# Installing your Python package

David J. Birnbaum
http://www.obdurodon.org, djbpitt@gmail.com

We begin this notebook with code that will reset the environment whenever we run all cells from the top. This is housekeeping, and not part of the tutorial.

In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Running a Python script

When you develop a Python application you can run it in a few different ways. Assuming you’re at the command line in the directory that contains your `filename.py` program and you want to use Python 3:

1. You can run it by invoking the Python interpreter explicitly: `python filename.py`.
1. You can make it executable (so that you can run it by just typing the filename, without having to precede it with `python`) by typing `chmod +x filename.py`. Once you’ve done that, you can run it from the command line with `./filename.py`. If you do this, you must begin the file with a shebang:
```python
#! /usr/bin/env python3
```
so that the file will use your default Python 3 executable.

If you want to be able to run the program from any directory (not just the one in which it resides), you can add the shebang and then move the file into a directory that is specified in your system `path` variable, so that when you specify just the filename (e.g., `filename.py`), the system will find it by looking in all of the directories specified in the path.

But what happens when you want to be able to import parts of your Python code into another Python program? For example, suppose you don’t want to run your entire Python program, and you just want to import a function you’ve defined there for use in a different program. Running the program in one of the ways specified above won’t help with the import.

## Importing functions from Python modules into Python scripts

### Functions available in the standard Python distributions

#### Preliminary definitions: script and module

A Python **script** is a Python file with a filename that ends in `.py` that you typically run in one of the ways described above. An importable Python **module** is a Python file with a filenames that ends in `.py` that contains function definitions, and you typically import those definitions into a script to augment the core functions made available in the standard distribution. You may never run the entire module file, and its only reason for existence may be so that you can import functions defined in it into other programs.

#### Functions that are always available

The standard distribution comes with some functions that are always available, such as `len()`, which returns the length of an object (e.g., the number of characters in a string or the number of items in a list). Functions that are always available are called *builtins*. Fire up a command-line Python interpreter and try the following:

In [40]:
print("The length of the string 'koala' is:", len('koala'))
print("The length of the list ['koala', 'wombat'] is:", len(['koala', 'wombat']))

The length of the string 'koala' is: 5
The length of the list ['koala', 'wombat'] is: 2


We don’t have to import the `len()` function; it’s available by default in any Python shell.

#### Functions imported from modules that are part of standard Python distributions

Other functions have to be imported before they can be used. For example, the functions most commonly used to work with regular expressions are located in a module called `re`, and if you try to use them without first importing the module, they fail. The following attempt to use the `search()` function to check whether the word “hat” contains any lower-case vowel letter will generate an error message (your error message may be different from ours):

In [41]:
search('[aeiou]','hat')

NameError: name 'search' is not defined

The preceding generates an error message because the `search()` function is not always available, that is, it is not like `len()`. The `search()` function is part of the `re` (“regular expression”) module, so we have to address it as `re.search()`, that is, by prefixing it with the `re.` *namespace* to specify that it is part of the `re` module. But that works only if we’ve loaded the `re` module; unlike with `len()`, which is always available, the functions defined in the `re` module become available only once you’ve loaded that module explicitly.

To demonstrate the need for loading the `re` module, in the code below we deliberately unload it (using the `del` instruction) and we then try to use the `search()` function that is part of that module. In real life we would, of course, load the `re` module; we unload it here only to prove that if it isn’t loaded, it isn’t available. If we don’t have the `re` module loaded and we attempt to use the `search()` function that’s defined inside it, that attempt produces an error:

In [42]:
del re
re.search('a', 'hat')

NameError: name 're' is not defined

The reason we get an error is that functions within a module (and the module name itself) are not available until we import the module. If we do that, though, the function works:

In [43]:
import re
re.search('[aeiou]', 'hat')

<_sre.SRE_Match object; span=(1, 2), match='a'>

The code above reports that the search operation finds a match.

#### Taking stock: *builtins* vs *standard modules*

We can use functions like `len()` because they’re always available, and we can use functions like `re.search()` if we first import the `re` module, which is part of the standard Python distributions (modules that are part of the standard distributions are called *standard modules*). But what about functions contributed by the community?

#### Functions available in modules installed through PyPI

The Python community makes thousands of packages available at the [PyPI](https://pypi.python.org/pypi?%3Aaction=browse), the ‘Python Package Index’. You can install any of these from the command line with the command `pip install packagename`. For example, to install the [CollateX](https://pypi.python.org/pypi/collatex/2.1.0) collation toolkit, type `pip install collatex` at the command line. You can then import from CollateX the same way you import from `re`. Once you’ve installed a PyPI module with `pip`, you can import and use it in the same way as you import and use standard modules. But what about modules you write yourself, or others that are not distributed through PyPI?

## User functions

Much as we can import functions from modules that are part of the standard Python distributions (like `re`) or that we install from PyPI (like CollateX), we can import functions from modules that we write ourselves. But we can do that only if Python can find those modules. Python finds modules by consulting a search space, which by default is, in order (copied from https://docs.python.org/3/tutorial/modules.html):

1. The directory containing the input script (or the current directory when no file is specified).
1. PYTHONPATH (a list of directory names, with the same syntax as the shell variable PATH).
1. The installation-dependent default.

Builtins are always available, and both standard modules and those installed from PyPI with `pip` are placed into a location that is part of PYTHONPATH. Modules that we write ourselves can be made available, then, in at least five ways, the fifth of which is to be preferred for reasons described below: 

1. Put them in the current directory, that is, the directory where you’re developing the program that is going to use the functions defined in the module.
1. Add the directory in which they reside to PYTHONPATH permanently.
1. Add the directory in which they reside to PYTHONPATH temporarily.
1. Copy or move them manually into a directory included in your PYTHONPATH.
1. Install them automatically into a directory in your PYTHONPATH.

The first of these is suitable if you’re going to use the module in only one location, but if you want it to be available in more than one project, you don’t want to have to copy or move or link it to all of them. The second isn’t scalable because your PYTHONPATH will grow out of control over time in you keep adding directories to it. The third isn’t portable; if your script loads a module by first adding its system location to PYTHONPATH, other users may not have put that module into the same location on their systems (especially if they use different operating systems). The fourth is awkward because it’s manual, and if you want to distribute your module, many of your users may not even know that they have a PYTHONPATH, or how to find the directories it includes.

The fifth strategy, then, is the only one that is flexible, scalable, and portable. But in order to install a module automatically into a directory in your PYTHONPATH, you need to learn about Python packaging, which is the focus of the rest of this tutorial.

(Note: This tutorial relies heavily on information from both the standard Python documentation and http://www.siafoo.net/article/77.)

## Packaging and installing a module

### Sample module: the `lpad()` function

Let’s assume that we’ve written a module that a function to left-pad a number with zeroes to a particular length. For example, if you pass it a value like `10` and tell it to left-pad it to four places, it should return `0010`. There are lots of ways to do this in Python, but we’re going to spin our own. In real life we’d incorporate robust error trapping (e.g., what if the arguments aren’t numbers? what if the input is longer than the return length after padding?), and we’d make our code more elegant and less clunky, but we’ll sacrifice robustness and elegance for brevity and clarity and offer the following, which assumes that both of the arguments to the `lpad()` function are non-negative integers and that the length after padding will be longer than the string length of the input value:

In [80]:
def lpad(input_value, length):
    input_string = str(input_value)
    padding = '0' * length
    result = str(padding + input_string)
    return result[-(len(result) - len(input_string)):]
lpad(10,4)  

'0010'

In the example above, the `lpad()` function takes two non-negative integer arguments, called `input_value` and `length`. It converts `input_value` to a string and uses `length` to construct a padding string of “0” values that is longer than needed. It sets `result` equal to the concatenation of those two strings and slices it to keep the number of characters on the right end of the string that is equal to the value of `length`. When we ask this function to left-pad the integer “10” to four places, it returns “0010”.

We can use this function without fuss as long as it’s in a file that’s part of the project we’re developing, but suppose we want it to be available in any project we might write, much as `len()` or `re.search()` are. In that case we need to package it and then install it automatically into a directory in our PYTHONPATH. Here’s how to do that.

### Structuring a package

Create a working directory that contains a file called `setup.py` and a directory that will hold the functions you want to make accessible in your module. In this case we’ll call our directory `padding`, under the assumption that eventually it might contain other padding functions. The `padding` directory must contain at least two things: a file called `__init__.py` and a file that contains the functions you want to make available (we’ll call it `arbitrary.py` to emphasize that the filename doesn’t matter). Once these pieces are all in place, you (or others) can change into the directory that holds `setup.py` and run:

```python
python setup.py install
```

This will install the functions in a directory in your PYTHONPATH, which will make it possible to import them into any Python program that you create anywhere on your system. We describe how to do that below.

### `setup.py`

The `setup.py` file looks like the following:

```python
from distutils.core import setup
setup(
    name='padding',
    packages=['padding'],
    description='Padding functions'
)
```

The `name` and `packages` properties are required; `description` is optional, and there are other optional properties (author, URL, license, dependencies, etc.). The value of the `name` argument is arbitrary for our purposes, but if you were to create an *egg* (a way of distributing Python packages), this would be the name it would be given. The `packages` argument is a list of directories relative to the location of the `setup.py` file, and each location must contain an `__init__.py` file and a file with a `.py` extension that contains the functions you want to make available. You can include multiple directories in this list, but in the simplest case, there’s just one.

### `__init__.py`

Every directory listed in the `packages` property within `setup.py` must contain an `__init__.py` file. The `__init__.py` file for our project looks as follows:

```python
from padding.arbitrary import lpad
__all__ = ['lpad']
```

The `from` line specifies the directory name followed by the filename that contains the functions, and you can then specify which functions to import. The `__all__` line lists the functions that will be imported if you use syntax like:

```python
from padding import *
```

This means that you can include functions in your file that are not exposed on input.

Note that endusers never see the name `arbitrary`. They need to know the package name (`padding`) in order to construct the `import` statement and they need to know the function name in order to use the function once they’ve imported the package, but the name of the module that contains the function is private to the setup process.

### `arbitrary.py`

Our `arbitrary.py` file contains the following:

```python
def lpad(input_value, length):
    input_string = str(input_value)
    padding = '0' * length
    result = str(padding + input_string)
    return result[-(len(result) - len(input_string)):]
```

In other words, it’s just the function definition. 

## Putting it all together

Once you’ve created the structure outlined above, change into the directory that contains your `setup.py` file and run `python setup.py install`. Then move into a completely different directory (to verify that you aren’t accidentally finding your padding function because it happens to be in the current working directory), fire up a Python console, and try:

In [82]:
import padding
padding.lpad(10,4)

'0010'

Alternatively, you can use:

In [83]:
from padding import *
lpad(10,4)

'0010'